In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
with open ("../data/baking.json", "r") as f:
    baking_recipes = json.load(f)

In [3]:
df = pd.DataFrame(baking_recipes)

In [4]:
nut_df = pd.DataFrame(list(df["nutrients"]))

# remove grams unit and convert it into float type
for col in nut_df.columns:
    nut_df[col] = nut_df[col].str.replace("g", "").astype("float")

In [ ]:
time_df = pd.DataFrame(list(df["times"]))

# calculate preparation time (mins) (need to first extract the hours and mins out first)
def cal_time(df, col):
    dum_df = df.copy()
    dum_df[col] = dum_df[col].replace("No Time", "").fillna("")
    dum_df["Hours_" + col] = dum_df[col].str.extract(r'(\d+) hr').fillna(0).astype(int) # extract number of hours using regex
    dum_df["Minutes" + col] = dum_df[col].str.findall(r'(\d+)\s*mins').apply(lambda x: int(x[-1]) if x else 0).astype(int) # extract the last number for mins
    dum_df[col + "_mins"] = dum_df["Hours_" + col] * 60 + dum_df["Minutes" + col] 
    result = dum_df[col + "_mins"].to_list()

    return result


time_df["prep_mins"] = cal_time(time_df, "Preparation")
time_df["cook_mins"] = cal_time(time_df, "Cooking")
time_df["total_mins"] = time_df["prep_mins"] + time_df["cook_mins"] # total time spent for cooking
time_df = time_df.drop(columns = ["Preparation", "Cooking"])

In [6]:
# create a new dataframe
new_df = pd.concat([df, time_df], axis= 1).drop(columns = ["nutrients", "times"])
new_df = pd.concat([new_df, nut_df], axis= 1)

In [12]:
new_df.to_json("../data/baking_cleaned.json",orient = "records")